In [ ]:
import boto3

# Definir nombres del bucket y archivos
BUCKET_NAME = "data-lake-x"
RAW_FILE = "bronze/clientes.csv"
PROCESSED_FILE = "silver/clientes_limpios.csv"

# Crear cliente S3
s3 = boto3.client("s3")

def lambda_handler(event, context):
    try:
        print("Iniciando procesamiento de archivos en S3...")

        # Verificar si el bucket existe
        s3.head_bucket(Bucket=BUCKET_NAME)
        print(f"El bucket {BUCKET_NAME} existe.")

        # Descargar archivo desde S3
        response = s3.get_object(Bucket=BUCKET_NAME, Key=RAW_FILE)
        data = response["Body"].read().decode("utf-8").splitlines()

        # Procesar líneas: reemplazar "F" y "M", eliminar filas vacías
        cleaned_data = []
        for line in data:
            columns = line.split(",")  # Separar por columnas

            # Verificar si alguna columna está vacía
            if "" in columns:
                continue  # Omitir la fila si tiene una celda vacía

            # Reemplazar valores en la columna de género
            updated_columns = [col.replace("F", "Femenino").replace("M", "Masculino") for col in columns]
            cleaned_data.append(",".join(updated_columns))

        # Convertir de nuevo a string
        output_data = "\n".join(cleaned_data)

        # Subir el nuevo archivo procesado a S3 en la carpeta "silver"
        s3.put_object(Bucket=BUCKET_NAME, Key=PROCESSED_FILE, Body=output_data)

        print(f"Archivo procesado subido a {PROCESSED_FILE}")

        return {
            "statusCode": 200,
            "body": f"Archivo procesado y guardado en {PROCESSED_FILE}"
        }

    except s3.exceptions.NoSuchBucket:
        print(f"Error: El bucket {BUCKET_NAME} no existe.")
        return {
            "statusCode": 404,
            "error": f"El bucket {BUCKET_NAME} no existe. Verifica el nombre y la región."
        }

    except Exception as e:
        print(f"Error procesando archivo: {str(e)}")
        return {
            "statusCode": 500,
            "error": str(e)
        }
